In [4]:
import pandas as pd
from datetime import datetime

def read_csv_file(file_path):
    tasks_df = pd.read_csv(file_path)
    tasks_df['due_date'] = pd.to_datetime(tasks_df['due_date'], dayfirst=True)  # Convert due_date to datetime objects
    return tasks_df


In [14]:
def separate_completed_tasks(tasks_df):
    completed_tasks = tasks_df[tasks_df['status'] == 'Done']
    remaining_tasks = tasks_df[tasks_df['status'] != 'Done']
    return completed_tasks, remaining_tasks

def save_completed_tasks(completed_tasks, file_path):
    try:
        existing_tasks = pd.read_csv(file_path)
        completed_tasks = pd.concat([existing_tasks, completed_tasks])
    except FileNotFoundError:
        pass  # If the file doesn't exist, we just write the completed_tasks to a new file
    completed_tasks.to_csv(file_path, index=False)

def prioritize_tasks(remaining_tasks):
    # priority_scores = {'low': 1, 'medium': 2, 'high': 3}
    remaining_tasks['priority_score'] = remaining_tasks['priority']
 
    # For 'due_date_score', the sooner the due date, the higher the score
    latest_due_date = remaining_tasks['due_date'].max()
    remaining_tasks['due_date_score'] = (latest_due_date - remaining_tasks['due_date']).dt.days

    remaining_tasks['total_score'] = remaining_tasks['priority_score'] + remaining_tasks['due_date_score']
    prioritized_tasks = remaining_tasks.sort_values('total_score', ascending=False)
    
    # Combine the 'title' and 'description' columns
    prioritized_tasks['title'] = prioritized_tasks.apply(lambda row: f"{row['title']} - {row['description']}", axis=1)

    prioritized_tasks['due_date'] = prioritized_tasks['due_date'].dt.strftime('%d-%m-%y')

    # Remove the 'description' column
    prioritized_tasks = prioritized_tasks.drop(columns=['description'])
    print(prioritized_tasks)
    # Remove the score columns before returning the sorted dataframe
    prioritized_tasks = prioritized_tasks.drop(columns=['priority_score', 'due_date_score', 'total_score'])

    return prioritized_tasks


def save_prioritized_tasks(prioritized_tasks, file_path):
    prioritized_tasks.to_csv(file_path, index=False)


In [15]:
def main(input_file_path, completed_file_path, output_file_path):
    tasks_df = read_csv_file(input_file_path)
    completed_tasks, remaining_tasks = separate_completed_tasks(tasks_df)
    save_completed_tasks(completed_tasks, completed_file_path)
    prioritized_tasks = prioritize_tasks(remaining_tasks)
    save_prioritized_tasks(prioritized_tasks, output_file_path)


In [17]:
main("task_list.csv", "completed_tasks.csv", "prioritized_tasks.csv")

                                              title      status  due_date  \
1                              Admin - Chris letter  incomplete  21-05-23   
4                           MT App - write up notes  incomplete  21-05-23   
5                           MT App - redo flow char  incomplete  21-05-23   
0                      Admin - Juan/Dan/Abhay/Reiss  incomplete  22-05-23   
6                  MT App - code modules:WA/QA/QUES  incomplete  22-05-23   
2                    D class - oganise lesson plan   incomplete  22-05-23   
22                          Danny - Beanstalk notes  incomplete  23-05-23   
7                             Project - Youtube app  incomplete  22-05-23   
14                       Greek App - write up notes  incomplete  22-05-23   
12                   Matthieu - Case study refactor  incomplete  24-05-23   
3                   D class - Send pano dance lists  incomplete  22-05-23   
13                            Greek App - send repo  incomplete  22-05-23   

In [17]:

read_csv_file("task_list.csv")